In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
#from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE

from rbot import BinanceConfig

In [3]:
import bokeh
bokeh.__version__

'3.2.2'

In [4]:
config = BinanceConfig.TESTSPOT("BTCUSDT")

binance = BinanceMarket(config)
binance.download(20)
binance

thread is already started


start,end
1693353600001000,1695124878366000
"""2023-08-30T00:00:00.001000""","""2023-09-19T12:01:18.366000"""
days=,20


In [11]:
binance

start,end
1693353600001000,1695124206585000
"""2023-08-30T00:00:00.001000""","""2023-09-19T11:50:06.585000"""
days=,20


In [13]:
ohlcv = binance.ohlcv(DAYS_BEFORE(20), 0, 60*60)
ohlcv

,open,high,low,close,volume
timestamp,,,,,
2023-08-30 12:00:00+00:00,27356.60,27425.00,27292.24,27353.64,1419.824590
2023-08-30 13:00:00+00:00,27353.64,27464.45,27145.23,27376.07,3215.906830
2023-08-30 14:00:00+00:00,27376.07,27424.40,27017.24,27110.01,4624.915250
2023-08-30 15:00:00+00:00,27110.00,27293.90,27036.66,27201.63,2931.789620
2023-08-30 16:00:00+00:00,27201.62,27210.84,27067.90,27155.37,1682.431670
...,...,...,...,...,...
2023-09-19 08:00:00+00:00,26299.99,45000.00,12410.00,26389.00,17.227446
2023-09-19 09:00:00+00:00,26389.00,27305.00,26178.00,26838.00,8.220878
2023-09-19 10:00:00+00:00,26787.13,50000.00,10000.00,26734.00,16.703096


In [14]:
ohlcv.columns

Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')

In [31]:

from collections import OrderedDict
import numpy as np
import pandas as pd

from bokeh.layouts import column 
from bokeh.models import ColumnDataSource, RangeTool, HoverTool, CrosshairTool, Span
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import datetime


class Chart:
    def __init__(self, width, height, ohlcv):
        output_notebook()        
        
        self.figure = OrderedDict()
        self.width = width
        self.x_range = None

        ######### create main price figure ############
        TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

        # setup main figure
        dates = np.array(ohlcv.index, dtype=datetime.datetime)        
        price = figure(x_axis_type="datetime", tools=TOOLS, width=self.width, height=height,
           title="Price", background_fill_color="#efefef", x_range=(dates[0], dates[-1]))
        
        self.x_range = price.x_range
        self.figure['price'] = price
        
        self.draw_ohlc(price, ohlcv)

        span_height = Span(dimension="height", line_dash="dashed", line_width=1)
        self.cross_hair = CrosshairTool(overlay=span_height)        
        price.add_tools(self.cross_hair)

        ########  create volume figure ################
        if 'volume' in ohlcv.columns:
            volume = self.new_figure('Volume', 100, 'Volume')
            self.draw_volume('Volume', ohlcv)
            volume.add_tools(self.cross_hair)

        ######### setup select figure #################
        """ Disable range tools
        select = figure(title="Price slide bar",
                height= int(height/4), width=self.width, y_range=price.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")            

        self.select = select

        self.line(select, ohlcv, x_key='timestamp', y_key='close', legend_label='price', color='#1010ff')
        
        range_tool = RangeTool(x_range=self.x_range)
        range_tool.overlay.fill_color = "navy"
        range_tool.overlay.fill_alpha = 0.2

        #select.ygrid.grid_line_color = None
        #select.add_tools(range_tool)
        #select.toolbar.active_multi = range_tool
        #select.add_tools(self.cross_hair)
        """
    
    def new_figure(self, name, height, title):
        p = figure(x_axis_type="datetime", width=self.width, height=height, tools="", toolbar_location=None,
            title=title, background_fill_color="#efefef", x_range=self.x_range)
        self.figure[name] = p
        p.add_tools(self.cross_hair)

        return p 
        
    def get_figure(self, figure):
        if isinstance(figure, str):
            if figure in self.figure:
                return self.figure[figure]
        
            return self.new_figure(figure, 100, figure)
        else:
            return figure

    def show(self):
        figure = []
        for key in self.figure:
            figure.append(self.figure[key])

        # remove select figure
        #figure.append(self.select)
            
        show(column(figure))

    def draw_result(self, df):
        buy_df = df[(df['order_side'] == "Buy") & (df['status'] != 'Expire')]
        buy_df_e = df[(df['order_side'] == "Buy") & (df['status'] == 'Expire')]
        sell_df = df[(df['order_side'] == "Sell") & (df['status'] != 'Expire')]
        sell_df_e = df[(df['order_side'] == "Sell") & (df['status'] == 'Expire')]

        p = self.new_figure('profit', 150, 'profit')
        self.step(p, df, 'update_time', 'sum_profit', legend_label='profit', color="#ff8080")

        p = self.new_figure('position', 100, 'position')
        self.step(p, df, 'update_time', 'position', legend_label='position', color="#ff8080")

        p = self.get_figure('price')
        self.draw_order_maker(p, buy_df, 'triangle', fill_color="#00ff00", line_color="#00ff00", legend_name='Buy')
        self.draw_order_maker(p, sell_df, 'inverted_triangle', fill_color="#ff0000", line_color="#ff0000", legend_name='Sell')
        self.draw_order_maker(p, buy_df_e, 'triangle', fill_color="#00000000", line_color="#00ff00", legend_name='Buy Expire')
        self.draw_order_maker(p, sell_df_e, 'inverted_triangle', fill_color="#00000000", line_color="#ff0000", legend_name='Sell Expire')


    def draw_ohlc(self, p, ohlc):
        ds = ColumnDataSource(ohlc)

        df_inc = ColumnDataSource(ohlc[(ohlc['open'] <= ohlc['close'])])
        df_dec = ColumnDataSource(ohlc[(ohlc['close'] < ohlc['open'])])

        delta = (ohlc[1:2].index - ohlc[0:1].index)[0]
        w = delta.total_seconds() * 1_000 * 0.8

        p.segment('timestamp', 'high', 'timestamp', 'low', source=ds, color="#080808")
        vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec, fill_color='#ff66ff', line_color='#ff0000', line_width=0)        
        vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc, fill_color='#66ccff', line_color="#10ff80", line_width=0)

        hover_inc = HoverTool(
            renderers=[vbar_inc],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime",
            },
            mode="vline",
            show_arrow=False,
        )       

        hover_dec = HoverTool(
            renderers=[vbar_dec],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime"
            },
            show_arrow=False,
        )      

        p.add_tools(hover_inc)
        p.add_tools(hover_dec)


    def draw_order_maker(self, p, df, marker, fill_color, line_color, legend_name, **kwargs):
        df = ColumnDataSource(df)
        scatter = p.scatter(x='update_time', y='order_price', source=df, marker=marker, size=12, fill_color=fill_color, line_color=line_color, line_width=1, legend_label=legend_name, **kwargs)

        hover = HoverTool(
            renderers=[scatter],
            tooltips = [
                (legend_name,""),
                ("timestamp", "@update_time{%F %R.%S}"),
                ("Price", "@order_price{0.0}")
            ],
            formatters={
                "@update_time": "datetime"
            }
        )
        p.add_tools(hover)    

    def draw_volume(self, figure, ohlcv):
        p = self.get_figure(figure)        
        self.line(p, ohlcv, x_key='timestamp', y_key='volume', color='#00ffff', legend_label='volume')

    
    def line2(self, figure, df, x_key, y_key, legend_label, color, **kwargs):
        p = self.get_figure(figure)
        df = self.make_df_from_series(df)        

        p.line(x=x_key, y=y_key, source=df, line_color=color, legend_label=legend_label, **kwargs)

    def line(self, figure, df, x_key= None, y_key=None, **kwargs):
        p = self.get_figure(figure)
        df = self.make_df_from_series(df)        
        
        if not x_key:
            x_key = 'timestamp'
        
        if not y_key:
            y_key = 'value'

        p.line(x=x_key, y=y_key, source=df, **kwargs)

    
    def step(self, figure, df, x_key=None, y_key=None, **kwargs):
        p = self.get_figure(figure)
        df = self.make_df_from_series(df)

        if not x_key:
            x_key = 'timestamp'
        
        if not y_key:
            y_key = 'value'

        p.step(x=x_key, y=y_key, source=df, mode='after', **kwargs)

    def make_df_from_series(self, df):
        return ColumnDataSource(self.make_df(df))
    
    def make_df(self, df):
        if isinstance(df, pd.DataFrame):
            df = df
        elif isinstance(df, pd.Series):
            df = df.rename('value')
            df = pd.DataFrame(df)
            df.index.name = 'timestamp'

            df.index = pd.to_datetime(df.index, utc=True, unit='us')
            df = df.dropna()
    
        df = df[~df.index.duplicated(keep='last')]
        
        return df



In [32]:
chart = Chart(900, 400, ohlcv)
chart.show()

Loading BokehJS ...

In [6]:


class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            super().__init__()
            self.K = param_K  # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*10
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        if session.short_order_size or session.long_order_size: #全開の注文処理中の場合はなにもしない       
            return 
        
        ohlcv_df = session.ohlcv(60*60*2, 6)       # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean() * self.K     #　過去５本足のレンジの平均値 * K

        # Long/Short判定
        diff_low   =   (ohlcv_latest["open"][0] - ohlcv_latest["low"][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest["open"][0] - ohlcv_latest["high"][0])  
        detect_long = range_width  < diff_high
        
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range', time_us, range_width)

        #　執行方法（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            if not session.long_position_size:
                if session.short_position_size:
                    session.place_order("Buy", session.best_buy_price, 0.02, 60*30, "doten Long")                        
                else:
                    session.place_order("Buy", session.best_buy_price, 0.01, 60*30, "Open Long")    
            else:
                pass

        if detect_short:
            if not session.short_position_size:
                if session.long_position_size:
                    session.place_order("Sell", session.best_sell_price, 0.02, 60*30, "Doten Short")                     
                else:
                    session.place_order("Sell", session.best_sell_price, 0.01, 60*30, "Open Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [7]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%

In [8]:
agent = Agent()
back_tester.run(agent, rbot.DAYS_BEFORE(10), 0)


,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-05 00:50:00+00:00,2022-12-05 00:50:56.656000+00:00,0000-0001,0,Buy,True,Open,17219.16,0.01,172.1916,0.00,...,17219.16,0.01,172.1916,0.0000,0.172192,-0.172192,0.01,Open Long,-0.172192,0.01
2022-12-05 15:10:00+00:00,2022-12-05 15:40:00.146000+00:00,0000-0002,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17207.99,0.02,344.1598,0.0000,0.000000,0.000000,0.00,Doten Short,-0.172192,0.01
2022-12-05 16:00:00+00:00,2022-12-05 16:08:35.636000+00:00,0000-0003,0,Sell,True,Close,17219.16,0.01,172.1916,17088.30,...,17088.30,0.01,170.8830,-1.3086,0.170883,-1.479483,-0.01,Doten Short,-1.651675,0.00
2022-12-05 16:00:00+00:00,2022-12-05 16:08:35.636000+00:00,0000-0003,1,Sell,True,Open,17088.30,0.01,170.8830,0.00,...,17088.30,0.01,170.8830,0.0000,0.170883,-0.170883,-0.01,Doten Short,-1.822558,-0.01
2022-12-06 23:40:00+00:00,2022-12-06 23:42:12.620000+00:00,0000-0004,0,Buy,True,Close,17088.30,0.01,170.8830,17067.54,...,17067.54,0.01,170.6754,0.2076,0.170675,0.036925,0.01,doten Long,-1.785633,0.00
2022-12-06 23:40:00+00:00,2022-12-06 23:42:12.620000+00:00,0000-0004,1,Buy,True,Open,17067.54,0.01,170.6754,0.00,...,17067.54,0.01,170.6754,0.0000,0.170675,-0.170675,0.01,doten Long,-1.956308,0.01
2022-12-07 07:20:00+00:00,2022-12-07 07:50:00.306000+00:00,0000-0005,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,16934.06,0.02,338.6812,0.0000,0.000000,0.000000,0.00,Doten Short,-1.956308,0.01
2022-12-09 13:40:00+00:00,2022-12-09 14:10:00.314000+00:00,0000-0006,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17233.61,0.02,344.6722,0.0000,0.000000,0.000000,0.00,Doten Short,-1.956308,0.01
2022-12-11 20:30:00+00:00,2022-12-11 20:47:03.013000+00:00,0000-0007,0,Sell,True,Close,17067.54,0.01,170.6754,17136.00,...,17136.00,0.01,171.3600,0.6846,0.171360,0.513240,-0.01,Doten Short,-1.443068,0.00


In [9]:
back_tester

Exchange name,BN
Market name,BTCBUSD
Size in price currency,False
maker fee rate,0.1 [%]
Agent class name,Agent
enable: on_tick,False
enable: on_clock,True
enable: on_update,True
clock interval:,600 [sec]
start,"2022-12-04T04:24:57.359000 (1,670,127,897,359,000)"
end,"2022-12-12T23:59:59.684000 (1,670,889,599,684,000)"


In [9]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_tester.result)

chart.show()

Loading BokehJS ...

ValueError: failed to validate Toolbar(id='p1487', ...).active_multi: expected an element of either Null, Auto or Instance(GestureTool), got RangeTool(id='p1521', ...)

In [ ]:
agent.get_indicator('diff_low')

,value
timestamp,
2022-12-04 04:10:00+00:00,0.09
2022-12-04 04:20:00+00:00,0.09
2022-12-04 04:30:00+00:00,0.52
2022-12-04 04:40:00+00:00,0.52
2022-12-04 04:50:00+00:00,0.52
...,...
2022-12-12 23:20:00+00:00,33.75
2022-12-12 23:30:00+00:00,33.75
2022-12-12 23:40:00+00:00,33.75


In [ ]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(r)

chart.new_figure("indicator_panel", 150, 'indicator')
chart.line('indicator_panel', agent.indicator('diff_low'), 'timestamp', 'value', 'diff_low', '#00ff00')
chart.line('indicator_panel', agent.indicator('diff_high'), 'timestamp', 'value', 'diff_high', '#ffff00')
chart.line('indicator_panel', agent.indicator('range'), 'timestamp', 'value', 'range', '#00ffff')

chart.show()

Loading BokehJS ...

price
volume
profit
position
indicator_panel
